<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import plotly.express as px
import os
import optuna
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from optuna.visualization.matplotlib import plot_param_importances

In [160]:
working_dir = os.getcwd()
data_path = working_dir + '/data/'
processed_path = data_path + 'processed/'
model_path = working_dir + '/models/'
optuna_path = working_dir + '/optuna_studies/'

In [161]:
#load our training and testing data
X = pd.read_csv(processed_path + 'X.csv', index_col='Id')
y = np.load(processed_path + 'y.npy')

In [162]:
#loading our preprocess pipeline
with open(model_path + 'preprocess_pipeline.h5', 'rb') as f:
    preprocessor = pickle.load(f)

In [163]:
print(preprocessor)

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='constant')),
                                                 ('quantile_transformer',
                                                  QuantileTransformer(n_quantiles=700,
                                                                      output_distribution='normal',
                                                                      random_state=42))]),
                                 Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFin...
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'Ga

In [164]:
X.shape

(1460, 85)

In [165]:
testing_set.shape

(1459, 85)

In [169]:
#fit and transform the training data
X = preprocessor.fit_transform(X)

In [8]:
#the Kaggle competition scores your entry based on RMSE
#so this is the function we will optimize with our Optuna study
def rmse(y_actual, y_predicted):
    return mean_squared_error(y_actual, y_predicted, squared=False)

In [22]:
#define optuna objective
def objective(trial):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8,test_size = 0.2, random_state = 42)
    
    #hyperparameters to optimize with optuna
    params = {
                 "booster": trial.suggest_categorical('booster', ["gbtree"]),
                 "n_jobs": trial.suggest_categorical('n_jobs', [4]),
                 "n_estimators": trial.suggest_int('n_estimators', 100, 1000, 100),
                 "learning_rate": trial.suggest_float('learning_rate', 0.01, 0.5),
                 "subsample": trial.suggest_float('subsample', 0.1, 0.5),
                 "colsample_bytree": trial.suggest_float('colsample_bytree', 0.1, 0.5),
                 "max_depth": trial.suggest_int("max_depth", 2, 20),
                 "reg_lambda": trial.suggest_float('reg_lambda', 2, 100),
                 "reg_alpha": trial.suggest_float('reg_alpha', 1, 50),
                'gamma': trial.suggest_loguniform('gamma', 1e-4,1e4),
                'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-4,1e4)
            }
    
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation_0-rmse')
    
    model = XGBRegressor(random_state = 42,
                         **params)
    model.fit(X_train, y_train,
              eval_set=[(X_valid, y_valid)],
              eval_metric=['rmse'],
              early_stopping_rounds = 10,
              verbose=2,
             callbacks =[pruning_callback])
    return rmse(y_valid, model.predict(X_valid))

In [23]:
#create Optuna study
study = optuna.create_study(direction='minimize', study_name = 'XGBRegressor')
study.optimize(objective, timeout=120*60)

best_trial = study.best_trial
print('Best root mean squared error: {}'.format(best_trial.value))
print('Best trial\'s parameters: ')
for key, value in best_trial.params.items():
    print('{}: {}'.format(key, value))
# Showing optimization results
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2021-10-23 15:31:01,089] A new study created in memory with name: XGBRegressor


[0]	validation_0-rmse:162734.35938
[2]	validation_0-rmse:117875.93750
[4]	validation_0-rmse:90656.84375
[6]	validation_0-rmse:74127.70312
[8]	validation_0-rmse:63640.00391
[10]	validation_0-rmse:56804.94922
[12]	validation_0-rmse:51661.55078
[14]	validation_0-rmse:47729.91406
[16]	validation_0-rmse:45297.74609
[18]	validation_0-rmse:43828.80859
[20]	validation_0-rmse:41770.59766
[22]	validation_0-rmse:40435.96484
[24]	validation_0-rmse:38632.23828
[26]	validation_0-rmse:38308.25391
[28]	validation_0-rmse:37686.17969
[30]	validation_0-rmse:36488.73438
[32]	validation_0-rmse:35919.60156
[34]	validation_0-rmse:35596.52344
[36]	validation_0-rmse:35146.47266
[38]	validation_0-rmse:34409.33203
[40]	validation_0-rmse:33828.50391
[42]	validation_0-rmse:33511.76172
[44]	validation_0-rmse:33200.64844
[46]	validation_0-rmse:32905.01562
[48]	validation_0-rmse:32714.28711
[50]	validation_0-rmse:32389.04883
[52]	validation_0-rmse:31955.41406
[54]	validation_0-rmse:31491.15039
[56]	validation_0-rmse:

[I 2021-10-23 15:31:01,552] Trial 0 finished with value: 29953.217369142403 and parameters: {'n_estimators': 800, 'learning_rate': 0.20892265451619105, 'n_jobs': 4, 'subsample': 0.38513195041246095, 'colsample_bytree': 0.27326141940672544, 'max_depth': 7, 'booster': 'gbtree', 'reg_lambda': 26.913719615265503, 'reg_alpha': 24.64169898010541, 'gamma': 0.14656889584962093, 'min_child_weight': 21.044499498339892}. Best is trial 0 with value: 29953.217369142403.


[0]	validation_0-rmse:151144.54688
[2]	validation_0-rmse:103220.93750
[4]	validation_0-rmse:79262.39062
[6]	validation_0-rmse:66806.87500
[8]	validation_0-rmse:59570.52344
[10]	validation_0-rmse:55004.49219
[12]	validation_0-rmse:51773.35938
[14]	validation_0-rmse:48741.67188
[16]	validation_0-rmse:47060.51172
[18]	validation_0-rmse:45547.29297
[20]	validation_0-rmse:44067.36719
[22]	validation_0-rmse:42884.50391
[24]	validation_0-rmse:41739.59375
[26]	validation_0-rmse:41327.93359
[28]	validation_0-rmse:40790.43359
[30]	validation_0-rmse:39854.51953
[32]	validation_0-rmse:39159.15625
[34]	validation_0-rmse:38617.23828
[36]	validation_0-rmse:38276.75000
[38]	validation_0-rmse:37631.61719
[40]	validation_0-rmse:37173.50781
[42]	validation_0-rmse:36706.61328
[44]	validation_0-rmse:36201.34766
[46]	validation_0-rmse:35748.27734
[48]	validation_0-rmse:35608.82422
[50]	validation_0-rmse:35135.85938
[52]	validation_0-rmse:34827.00000
[54]	validation_0-rmse:34357.40625
[56]	validation_0-rmse:

[I 2021-10-23 15:31:02,027] Trial 1 finished with value: 30613.421250096235 and parameters: {'n_estimators': 300, 'learning_rate': 0.323125936587582, 'n_jobs': 4, 'subsample': 0.3782318077367197, 'colsample_bytree': 0.4172883296087002, 'max_depth': 2, 'booster': 'gbtree', 'reg_lambda': 81.7047296537935, 'reg_alpha': 12.180330017019319, 'gamma': 0.0008748867897591302, 'min_child_weight': 33.144558127080714}. Best is trial 0 with value: 29953.217369142403.


...

[0]	validation_0-rmse:114632.10156


[I 2021-10-23 17:30:57,129] Trial 10821 pruned. Trial was pruned at iteration 0.
[I 2021-10-23 17:30:58,537] Trial 10822 pruned. Trial was pruned at iteration 0.
[I 2021-10-23 17:30:59,911] Trial 10823 pruned. Trial was pruned at iteration 0.


[0]	validation_0-rmse:112507.89062


[I 2021-10-23 17:31:01,249] Trial 10824 pruned. Trial was pruned at iteration 1.


Best root mean squared error: 22639.86321674742
Best trial's parameters: 
n_estimators: 800
learning_rate: 0.4910429061295169
n_jobs: 4
subsample: 0.4314552475257108
colsample_bytree: 0.4999379628659259
max_depth: 2
booster: gbtree
reg_lambda: 3.7032254243585974
reg_alpha: 6.373633275926588
gamma: 0.0015642079516800683
min_child_weight: 0.05530311051359703
Number of finished trials: 10825
Best trial parameters: {'n_estimators': 800, 'learning_rate': 0.4910429061295169, 'n_jobs': 4, 'subsample': 0.4314552475257108, 'colsample_bytree': 0.4999379628659259, 'max_depth': 2, 'booster': 'gbtree', 'reg_lambda': 3.7032254243585974, 'reg_alpha': 6.373633275926588, 'gamma': 0.0015642079516800683, 'min_child_weight': 0.05530311051359703}
Best score: 22639.86321674742


In [31]:
#save our Optuna study
with open(optuna_path + f'XGBoost_study.pickle', 'wb') as f:
        pickle.dump(study, f)

In [179]:
#define best_model with optimized hyperparameters
best_params = study.best_trial.params 
best_model = XGBRegressor(random_state=42,
                         **best_params)

In [187]:
#finally we save our best XGBoost model
XGBoost_path = model_path + "XGBoost/"
with open(XGBoost_path + f'best_XGBoost.pickle', 'wb') as f:
        pickle.dump(best_model, f)